In [22]:
import numpy as np
import math
import cvxpy as cp
from itertools import combinations
import time
import copy
import pandas as pd
import copy
import mosek

In [23]:
np.random.seed(1)

In [24]:
def collection_distribution(n,collection):
    #collection = list(combinations(range(n),size))
    collection_distribution = np.zeros((n,len(collection)))

    for i in range(len(collection)):
        distribution_S = np.random.exponential(1, len(collection[i]))
        assortment_dis = np.array([k/np.sum(distribution_S) for k in distribution_S])
        for j in range(len(collection[i])):
            collection_distribution[collection[i][j]][i]= assortment_dis[j]
            
    return collection_distribution

In [25]:
n = 7
instance_size = 100
#collection_size = [2,3,5,10,15,20]
collection_size = [2,3,5,10,15]
#collection_size = [2,3,5]

''' instance_size = 1
collection_size = [2] '''

all_size2_ass = [list(x) for x in list(combinations(range(n),2))]
all_size3_ass = [list(x) for x in list(combinations(range(n),3))]
#all_size4_ass = [list(x) for x in list(combinations(range(n),4))]
#all_possible_assortment = all_size2_ass + all_size3_ass + all_size4_ass
all_possible_assortment = all_size2_ass + all_size3_ass 
print(len(all_possible_assortment))



all_collection = [[] for _ in range(len(collection_size))]

# construct 1000 samples of random assortment size collection of first collection size
for j in range(instance_size):
    collection = []
    while(len(collection)<collection_size[0]):
        S = all_possible_assortment[np.random.randint(len(all_possible_assortment))]
        if S not in collection:
            collection.append(S)
    all_collection[0].append(collection)


for i in range(1,len(collection_size)):
    all_collection[i] = copy.deepcopy(all_collection[i-1])
    for j in range(instance_size):
        while(len(all_collection[i][j])<collection_size[i]):
            S = all_possible_assortment[np.random.randint(len(all_possible_assortment))]
            if S not in all_collection[i][j]:
                all_collection[i][j].append(S)

all_instance = []
for i in range(len(collection_size)):
    collection_ins = []   
    for j in range(instance_size):
        collection_ins.append(collection_distribution(n,all_collection[i][j]))
        ''' v = np.random.normal(0,1,size=n) #standard normal
        curr_coll_mnl = collection_distribution_mnl(n,all_collection[i][j],v)
        collection_ins.append(gaussian_noise(curr_coll_mnl,std,alpha)) '''
    all_instance.append(collection_ins)


56


In [26]:
def mdm_limit(data,collection):
    eps = 0.00001
    n,m = data.shape
        
    ind = [[] for i in range(n)]  ## keeps tracking of the assortments including product i 
    for i in range(len(collection)):
        for j in collection[i]:
            ind[j].append(i) ## assortment i includes product j 
    ''' ind = []
    for i in range(n):
        idx = list(np.where(data[i]!=0)[0])
        ind.append(idx) '''
        
    lam = cp.Variable(shape=m,nonneg = True)
    x = cp.Variable((n,m),nonneg = True)
    delta = cp.Variable((m,m),boolean = True)
    y = cp.Variable((n,m),boolean = True)
    
    #add constraints
    #normalization conditions
    constraints = [sum(x[:,j])==1 for j in range(m)]
    
    # if product not in the assortment the choice probabilities should be zero
    for i in range(len(collection)):
        for j in range(n):
            if j not in collection[i]:
                constraints += [x[j][i]==0]
    
    for i in range(n):
        for j in ind[i]:
            #constraints += [x[i][j] + y[i][j] - 1 <=0]
            for k in ind[i]:
                if j!=k:
                    constraints += [x[i][j] - x[i][k] + delta[j][k]>=0]
                    constraints += [x[i][j] - x[i][k] -1 + delta[j][k]+eps*delta[j][k]<=0]
                    
                    constraints += [lam[j]-lam[k]+1 - delta[j][k]-eps*delta[j][k] >= 0]
                    
                    constraints += [lam[j]-lam[k]+y[i][j]+y[i][k]+delta[j][k]+delta[k][j]>=0]
                    constraints += [lam[j]-lam[k]-y[i][j]-y[i][k]-delta[j][k]-delta[k][j]<=0]
    for i in range(m):
        constraints += [lam[i]<=1]
    
    for i in range(len(collection)):
        for j in collection[i]:
            constraints += [x[j][i] + y[j][i]-1 <=0]
            #constraints += [x[j][i] - data[j][i]==0]
    
    #obj = cp.Minimize(sum(cp.norm1(x[:,j]-data[:,j]) for j in range(m)))
    obj = cp.Minimize(sum(data[i][j]*cp.abs(data[i][j] - x[i][j]) for i in range(n) for j in range(m)))
    
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.GUROBI)
    
    ''' print("\nstatus:", prob.status)
    print("\nThe optimal value is", prob.value)
    print("\nThe solution x is")
    print(x.value)
    print("\nThe solution lam is")
    print(lam.value)
    print("\nThe solution delta is")
    print(delta.value)
    print("\nThe solution y is")
    print(y.value) '''
    
    if prob.value < math.pow(10,-6):
        return [0,x.value,lam.value,prob.solver_stats.solve_time] 
    else:
        return [prob.value,x.value,lam.value,prob.solver_stats.solve_time]

In [56]:
def micp_mdm(data,collection):
    
    n,m = data.shape
    ''' prod_in_ass = []
    for i in range(n):
        idx = list(np.where(data[i]!=0)[0])
        prod_in_ass.append(idx) '''
        
    prod_in_ass = [[] for i in range(n)]  ## keeps tracking of the assortments including product i 
    for i in range(len(collection)):
        for j in collection[i]:
            prod_in_ass[j].append(i) ## assortment i includes product j 
    
    eps = 0.00001
        
    lam = cp.Variable(shape=m,nonneg = True)
    x = cp.Variable((n,m),nonneg = True)
    delta = cp.Variable((m,m),boolean = True)
    #y = cp.Variable((n,m),boolean = True)
    
    
    #add constraints
    #normalization conditions
    constraints = [sum(x[:,j])==1 for j in range(m)]
    
    # if product not in the assortment the choice probabilities should be zero
    for i in range(len(collection)):
        for j in range(n):
            if j not in collection[i]:
                constraints += [x[j][i]==0]
    
    ''' for i in range(len(prod_in_ass)):
        for j in prod_in_ass[i]:
            constraints += [x[i][j] + y[i][j] - 1 <=0] '''
    
    for i in range(len(prod_in_ass)):
        if len(prod_in_ass)>0:
            for j in prod_in_ass[i]:
                for k in prod_in_ass[i]:
                    if j !=k:
                        constraints += [lam[j] - lam[k] + delta[j][k]>=0]
                        constraints += [lam[j] - lam[k] -1 + delta[j][k]+eps*delta[j][k]<=0 ]
                        constraints += [x[i][j] - x[i][k] -delta[j][k]+1>=0]
                        constraints += [x[i][j] - x[i][k] -1 +delta[k][j]<=0]
                        constraints += [x[i][j] - x[i][k]  + delta[j][k]+delta[k][j]>=0]
                        constraints += [x[i][j] - x[i][k]  - delta[j][k]-delta[k][j]<=0]

    for i in range(m):
        constraints += [lam[i]<=1]
    
    #obj = cp.Minimize(sum(cp.norm1(x[:,j]-data[:,j]) for j in range(m)))
    obj = cp.Minimize(sum(data[i][j]*cp.abs(data[i][j] - x[i][j]) for i in range(n) for j in range(m)))
    
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.GUROBI)
    
    ''' print("\nstatus:", prob.status)
    print("\nThe optimal value is", prob.value)
    print("\nThe solution x is")
    print(x.value)
    print("\nThe solution lam is")
    print(lam.value)
    print("\nThe solution delta is")
    print(delta.value)
    print("\nThe solution y is")
    print(y.value) '''
    
    if prob.value < math.pow(10,-6):
        return [0,prob.solver_stats.solve_time] 
    else:
        return [prob.value,prob.solver_stats.solve_time]

In [28]:
print('starting testing the limit of MDM')
start_time = time.time()
mdm_runtime = []
mdm_total_loss = []  #of different collection size 
mdm_mean_loss = []
mdm_loss_se = []
mdm_runtime_se = []
for i in range(len(all_instance)):
    loss = []
    runtime = [] # of the same collection size
    for j in range(len(all_instance[i])):
        result = micp_mdm(all_instance[i][j],all_collection[i][j])
        #result = mdm_limit(all_instance[i][j],all_collection[i][j])
        loss.append(result[0])
        runtime.append(result[1])
        
    test = pd.DataFrame({'loss':loss,'runtime':runtime})
    test.to_csv('mdm/collection_'+str(collection_size[i])+'_old.csv')
    runtime = np.array(runtime)
    mdm_runtime.append(np.mean(runtime))
    mdm_runtime_se.append(np.std(runtime)/np.sqrt(len(runtime)))
    loss = np.array(loss)
    mdm_mean_loss.append(np.mean(loss))
    mdm_total_loss.append(np.sum(loss))
    mdm_loss_se.append(np.std(loss)/np.sqrt(len(loss)))
    
print('the total running time',time.time()-start_time)

starting testing the limit of MDM
the total running time 2628.891783952713


In [29]:
mdm = pd.DataFrame({'collection_size':collection_size,'mdm_mean_oss':mdm_mean_loss,'mdm_loss_se':mdm_loss_se,'mdm_mean_runtime':mdm_runtime,'mdm_runtime_se':mdm_runtime_se})
mdm.to_csv('mdm_limit_summary_old.csv')

In [30]:
mdm_mean_loss

[0.011784519336246585,
 0.057604606483913405,
 0.18719732911611284,
 0.781006547307558,
 1.5803082842022915]